In [40]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import matplotlib.pyplot as plt

import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np


In [41]:
#load the dataset
df = pandas.read_csv("/home/sakthy1497/Downloads/Amazon_Review.csv")

In [42]:
df.head()

,Comments,Sentiment,Intent,Context
0,So there is no way for me to plug it in here i...,negative,complaint,others
1,"Good case, Excellent value.",positive,feedback,quality
2,Great for the jawbone.,positive,feedback,quality
3,Tied to charger for conversations lasting more...,negative,complaint,quality
4,The mic is great.,positive,feedback,quality


In [43]:
df['Comments']

0       So there is no way for me to plug it in here i...
1                             Good case, Excellent value.
2                                  Great for the jawbone.
3       Tied to charger for conversations lasting more...
4                                       The mic is great.
                              ...                        
1995    I think food should have flavor and texture an...
1996                             Appetite instantly gone.
1997    Overall I was not impressed and would not go b...
1998    The whole experience was underwhelming, and I ...
1999    Then, as if I hadn't wasted enough of my life ...
Name: Comments, Length: 2000, dtype: object

## Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [44]:

df['Content_Parsed_1'] = df['Comments'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

In [45]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Punctuation signs

In [46]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

### 1.4. Possessive pronouns
We'll also remove possessive pronoun terminations:

In [47]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [48]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to /home/sakthy1497/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sakthy1497/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [50]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
Comments            2000 non-null object
Sentiment           2000 non-null object
Intent              2000 non-null object
Context             2000 non-null object
Content_Parsed_1    2000 non-null object
Content_Parsed_2    2000 non-null object
Content_Parsed_3    2000 non-null object
Content_Parsed_4    2000 non-null object
dtypes: object(8)
memory usage: 125.1+ KB


In [52]:
len(lemmatized_text_list)

2000

In [53]:
df['Content_Parsed_5'] = lemmatized_text_list

### 1.6. Stop words

In [54]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sakthy1497/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [56]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [57]:
df.head()

,Comments,Sentiment,Intent,Context,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,So there is no way for me to plug it in here i...,negative,complaint,others,So there is no way for me to plug it in here i...,so there is no way for me to plug it in here i...,so there is no way for me to plug it in here i...,so there is no way for me to plug it in here i...,so there be no way for me to plug it in here i...,way plug us unless go converter
1,"Good case, Excellent value.",positive,feedback,quality,"Good case, Excellent value.","good case, excellent value.",good case excellent value,good case excellent value,good case excellent value,good case excellent value
2,Great for the jawbone.,positive,feedback,quality,Great for the jawbone.,great for the jawbone.,great for the jawbone,great for the jawbone,great for the jawbone,great jawbone
3,Tied to charger for conversations lasting more...,negative,complaint,quality,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,tie to charger for conversations last more tha...,tie charger conversations last 45 minutesm...
4,The mic is great.,positive,feedback,quality,The mic is great.,the mic is great.,the mic is great,the mic is great,the mic be great,mic great


In [58]:
list_columns = ["Comments", "Intent", "Sentiment", "Context", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df.head()

,Comments,Intent,Sentiment,Context,Content_Parsed
0,So there is no way for me to plug it in here i...,complaint,negative,others,way plug us unless go converter
1,"Good case, Excellent value.",feedback,positive,quality,good case excellent value
2,Great for the jawbone.,feedback,positive,quality,great jawbone
3,Tied to charger for conversations lasting more...,complaint,negative,quality,tie charger conversations last 45 minutesm...
4,The mic is great.,feedback,positive,quality,mic great


## 2. Label coding
We'll create a dictionary with the label codification:

In [59]:
intent_codes = {
    'feedback' : 0,
    'complaint' : 1,
    'spam' : 2,
    'marketing' : 3
}

In [60]:
# Category mapping
df['Intent_Codes'] = df['Intent']
df = df.replace({'Intent_Codes':intent_codes})
df.head()

,Comments,Intent,Sentiment,Context,Content_Parsed,Intent_Codes
0,So there is no way for me to plug it in here i...,complaint,negative,others,way plug us unless go converter,1
1,"Good case, Excellent value.",feedback,positive,quality,good case excellent value,0
2,Great for the jawbone.,feedback,positive,quality,great jawbone,0
3,Tied to charger for conversations lasting more...,complaint,negative,quality,tie charger conversations last 45 minutesm...,1
4,The mic is great.,feedback,positive,quality,mic great,0


## 3. Train - test split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Intent_Codes'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [62]:
print(y_train, y_test)

385     1
1180    1
1407    0
624     0
1602    0
       ..
986     1
133     0
361     0
1364    0
451     0
Name: Intent_Codes, Length: 1700, dtype: int64 1859    1
1059    1
114     0
791     0
1813    1
       ..
1121    1
955     1
1073    1
1906    1
271     0
Name: Intent_Codes, Length: 300, dtype: int64


## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

In [63]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [64]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1700, 181)
(300, 181)


Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

In [65]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(intent_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


# 'complaint' category:
  . Most correlated unigrams:
. poor
. waste
. love
. good
. great
  . Most correlated bigrams:
. work great
. waste money

# 'feedback' category:
  . Most correlated unigrams:
. amaze
. nice
. good
. love
. great
  . Most correlated bigrams:
. waste money
. work great

# 'marketing' category:
  . Most correlated unigrams:
. item
. buy
. would
. highly
. recommend
  . Most correlated bigrams:
. go back
. work well

# 'spam' category:
  . Most correlated unigrams:
. new
. awesome
. night
. know
. stay
  . Most correlated bigrams:
. customer service
. work great



As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [66]:
bigrams

['go back',
 'come back',
 'sound quality',
 'work well',
 'waste money',
 'customer service',
 'work great']

We can see there are only seven. This means the unigrams have more correlation with the intent than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [67]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)